In [1]:
MAIN_PATH = "/home/carlos/PycharmProjects/tfm"
DATA_PATH = '{}/data/json'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [208]:
%load_ext autoreload
%autoreload 2
from scripts.text.article_text_processor import ArticleTextProcessor
from scripts.text.basic_text_processor import BasicTextProcessor
from scripts.text.teams_players import TeamPlayers
from scripts.text.key_events_summary import KeyEventsSummary

from scripts.conf import TEAMS

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Eventos importantes

El objetivo es realizar un resumen extractivo basandose en eventos importantes de un partido de fútbol. 
La idea es:

- Obtener eventos importantes: resultado, goles y ocasiones por ejemplo
- Buscar en la noticia frases relacionadas con esos eventos, juntarlas y formar el resumen

In [165]:
processor = ArticleTextProcessor()
key_events = KeyEventsSummary()

In [5]:
all_files = processor.load_json()

In [168]:
all_files.keys()

dict_keys(['italian_serie_a_2019_2020.json', 'champions_league_2019_2020.json', 'mls_2018_2019.json', 'spanish_la_liga_2019_2020.json', 'italian_serie_a_2017_2018.json', 'german_bundesliga_2017_2018.json', 'mls_2015_2016.json', 'german_bundesliga_2019_2020.json', 'mls_2016_2017.json', 'mls_2017_2018.json', 'spanish_la_liga_2017_2018.json', 'premier_league_2016_2017.json', 'german_bundesliga_2018_2019.json', 'italian_serie_a_2018_2019.json', 'premier_league_2019_2020.json', 'premier_league_2018_2019.json', 'french_ligue_one_2019_2020.json', 'spanish_la_liga_2018_2019.json', 'premier_league_2017_2018.json', 'mls_2019_2020.json'])

In [7]:
season_file = 'premier_league_2019_2020.json'
league_season_teams = TEAMS[season_file.split('.')[0]]

In [8]:
prueba = all_files[season_file]['https://www.bbc.com/sport/football/49791610']

In [34]:
key_words = ['goal', 'red', 'attempt']

In [54]:
events = prueba['events']

In [123]:
processed_events = key_events.process_match_events(events, keep_key_events=True)

Number of original events: 125
Number of processed events: 27


In [52]:
key_events.events_mapping_list

[28,
 29,
 34,
 35,
 44,
 45,
 47,
 48,
 50,
 54,
 59,
 62,
 63,
 64,
 65,
 68,
 71,
 72,
 77,
 78,
 82,
 97,
 110,
 113,
 116,
 117,
 118]

In [125]:
events[key_events.events_mapping_list[2]]

'Attempt saved. Nicolas Pépé (Arsenal) left footed shot from the centre of the box is saved in the bottom right corner. Assisted by Pierre-Emerick Aubameyang.'

In [126]:
processed_events[2]

'assisted shot saved Nicolas Pepe Pierre-Emerick Aubameyang corner box right arsenal footed attempt left'

__Buscando eventos en noticias__

Buscamos la frase más cercana a cada evento en cada artículo. Para ello, hacemos una aproximación
simple, usando un tfidf con el cuerpo de la noticia, y calculando la distancia coseno a cada
evento. 

Quizás este enfoque no funcione demasiado bien al ser las noticias tan cortas.

In [10]:
article = prueba['article']

In [49]:
article

'Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.\nIn a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.\nArsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang\'s cool finish. It was originally ruled out for offside but the video assistant referee confirmed the striker had been played onside by Harry Maguire.\nGoalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford\'s late free-kick, while Bukayo Saka\'s goalbound shot crucially struck Victor Lindelof and flew over the top.\nMcTominay also headed a great chance well over from Ashley Young\'s corner but neither side could force a winner.\nThe result takes Arsenal into th

Necesitamos guardar todas las frases:

In [68]:
doc_sents_list = [str(sent) for sent in key_events.text_proc.get_sentences(article)]

In [69]:
len(doc_sents_list)

39

Procesamos las frases del artículo de la misma forma que los eventos:

In [16]:
processed_sentences = key_events.process_match_article(article)

In [132]:
len(processed_sentences)

39

In [153]:
processed_sentences

['old league stalemate Manchester United trafford challenge Arsenal away far clubs grim illustrate provided evidence',
 'attritional shadow disappointing lead Manchester United edge drive Scott McTominay gone gave half years encounter rising mighty end',
 "Pierre-Emerick Aubameyang's arsenal equalised cool finish hour courtesy",
 'offside ruled video confirmed onside assistant originally striker Harry Maguire',
 "excelled Bernd Leno goalbound goalkeeper shot Marcus Rashford's saves crucially kick Victor Lindelof Arsenal flew Bukayo Saka's fine struck",
 "chance McTominay corner force headed Ashley Young's winner",
 'worst games start failing finished means seven Manchester United time double arsenal season takes years',
 'captains led demonstrated circumstances instantly sides clubs',
 "old newly united led manchester divisive supporters Ashley Young Aaron Wan-Bissaka's Granit Xhaka service absence appointed captain Arsenal defensive injury",
 'decisions led fashion knack showing numer

Para usar el tfidf de sklearn, necesitamos proporcionar un iterable de elementos (frases en este caso):

In [17]:
processed_sentences_wc = ' '.join(processed_sentences)

También se tienen en cuenta bigramas para no perder las entidades nombradas (quizás habría que probar
con trigramas también):

In [110]:
vectorizer = CountVectorizer(ngram_range=(1,2), strip_accents='unicode')

In [112]:
X = vectorizer.fit_transform(processed_sentences).toarray()

In [113]:
vectorizer.get_feature_names()

['aaron',
 'aaron wan',
 'absence',
 'absence appointed',
 'action',
 'action decision',
 'action evasive',
 'appeal',
 'appearing',
 'appearing mctominay',
 'appointed',
 'appointed captain',
 'arsenal',
 'arsenal away',
 'arsenal defensive',
 'arsenal equalised',
 'arsenal felt',
 'arsenal flew',
 'arsenal point',
 'arsenal provide',
 'arsenal season',
 'arsenal unai',
 'ashley',
 'ashley young',
 'assistant',
 'assistant originally',
 'assisted',
 'assisted onside',
 'attritional',
 'attritional shadow',
 'aubameyang',
 'aubameyang arsenal',
 'aubameyang touch',
 'away',
 'away build',
 'away far',
 'ball',
 'ball box',
 'bernd',
 'bernd leno',
 'best',
 'best action',
 'best bournemouth',
 'better',
 'better chances',
 'bissaka',
 'bissaka granit',
 'blocks',
 'boss',
 'boss champions',
 'bournemouth',
 'bournemouth points',
 'box',
 'build',
 'build getting',
 'bukayo',
 'bukayo saka',
 'captain',
 'captain arsenal',
 'captains',
 'captains led',
 'catch',
 'catch manchester',
 'c

In [24]:
X.shape

(39, 520)

In [114]:
tfidfconverter = TfidfTransformer()

In [115]:
X = tfidfconverter.fit_transform(X).toarray()

In [116]:
X.shape

(39, 520)

Eventos

In [144]:
X_events = vectorizer.transform(processed_events)

In [145]:
X_events = tfidfconverter.transform(X_events).toarray()

In [30]:
X_events.shape

(27, 520)

In [91]:
X_events

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [147]:
count_ixs = np.argwhere(X_events > 0)

In [148]:
import pandas as pd
import numpy as np

In [149]:
count_ixs

array([[  0,  52],
       [  0, 100],
       [  0, 270],
       [  0, 291],
       [  0, 295],
       [  0, 374],
       [  0, 414],
       [  0, 477],
       [  1,  12],
       [  1,  26],
       [  1,  52],
       [  1,  55],
       [  1,  56],
       [  1, 270],
       [  1, 343],
       [  1, 380],
       [  1, 414],
       [  2,  12],
       [  2,  26],
       [  2,  30],
       [  2,  52],
       [  2, 100],
       [  2, 139],
       [  2, 140],
       [  2, 270],
       [  2, 343],
       [  2, 351],
       [  2, 352],
       [  2, 374],
       [  2, 414],
       [  3,  52],
       [  3, 291],
       [  3, 295],
       [  3, 374],
       [  3, 414],
       [  3, 477],
       [  4,  26],
       [  4,  36],
       [  4,  52],
       [  4, 270],
       [  4, 291],
       [  4, 295],
       [  4, 296],
       [  4, 297],
       [  4, 370],
       [  4, 414],
       [  4, 477],
       [  5,  26],
       [  5,  52],
       [  5, 291],
       [  5, 295],
       [  5, 374],
       [  5,

In [ ]:
[  1,  12],
       [  1,  26],
       [  1,  52],
       [  1,  55],
       [  1,  56],
       [  1, 270],
       [  1, 343],
       [  1, 380],
       [  1, 414],

In [140]:
processed_events[1]

'outside wide shot high assisted Nicolas Pepe attempt box arsenal Bukayo Saka footed missed left'

In [150]:
pd.DataFrame(X_events, columns=vectorizer.get_feature_names()).iloc[1, [12,26,52,55,56,270,343,380,414]]

arsenal        0.207616
assisted       0.362105
box            0.362105
bukayo         0.325361
bukayo saka    0.362105
left           0.362105
pepe           0.325361
saka           0.299290
shot           0.362105
Name: 1, dtype: float64

In [152]:
doc_sents_list[4]

"Goalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford's late free-kick, while Bukayo Saka's goalbound shot crucially struck Victor Lindelof and flew over the top.\n"

Distancia entre eventos y frases:

In [133]:
distances = cosine_similarity(X_events, X)

In [134]:
distances.shape

(27, 39)

In [151]:
distances[1]

array([0.02357063, 0.        , 0.0296338 , 0.        , 0.23169911,
       0.        , 0.02319782, 0.        , 0.01946064, 0.        ,
       0.        , 0.        , 0.        , 0.0412068 , 0.02066832,
       0.05721444, 0.        , 0.07172153, 0.        , 0.07333465,
       0.        , 0.17373423, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04316312, 0.        , 0.        ,
       0.        , 0.03889747, 0.        , 0.        , 0.11114276,
       0.        , 0.3253609 , 0.15101771, 0.        ])

In [136]:
distances.max(axis=1)

array([0.12384918, 0.3253609 , 0.3729923 , 0.14625438, 0.21650615,
       0.13250492, 0.3253609 , 0.20174099, 0.20161512, 0.42301245,
       0.14752496, 0.25041393, 0.38048124, 0.41625515, 0.34408246,
       0.17442482, 0.22877419, 0.14625438, 0.3473272 , 0.21650615,
       0.21457486, 0.13485292, 0.14752496, 0.24143053, 0.23138639,
       0.1796058 , 0.16062666])

In [137]:
sentences_ixs = distances.argmax(axis=1)

In [138]:
sentences_ixs

array([ 0, 36,  2,  0,  4,  0, 36, 13,  1,  2, 34,  1,  2,  2, 36, 21,  4,
        0,  5,  4,  3,  0, 34,  5,  4, 21, 21])

In [167]:
processed_events

['corner right left shot Manchester United saved box footed Andreas Pereira attempt',
 'high outside left shot arsenal Nicolas Pépé Bukayo Saka wide missed box footed assisted attempt',
 'corner right left shot arsenal Nicolas Pépé Pierre-Emerick Aubameyang saved box footed assisted attempt',
 'right outside shot blocked Paul Pogba Manchester United box footed attempt',
 'ball left shot blocked Paul Pogba Manchester United box footed assisted Marcus Rashford attempt',
 'right shot blocked Manchester United Jesse Lingard box footed assisted Andreas Pereira attempt',
 'corner right left shot arsenal Bukayo Saka saved box footed attempt',
 'corner right angle shot arsenal difficult saved footed attempt Matteo Guendouzi',
 'corner right outside left shot arsenal goal Manchester United Scott McTominay box footed assisted Marcus Rashford',
 'right shot lucas arsenal Pierre-Emerick Aubameyang misses missed box footed assisted attempt cross',
 'right outside blocked shot arsenal David Luiz box

In [58]:
doc_sents_list

[Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.,
 In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.,
 Arsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.,
 It was originally ruled out for offside but the video assistant referee confirmed the striker had been played onside by Harry Maguire.,
 Goalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford's late free-kick, while Bukayo Saka's goalbound shot crucially struck Victor Lindelof and flew over the top.,
 McTominay also headed a great chance well over from Ashley Young's corner but neither side could force a winner.,
 The result takes Arsenal into

In [139]:
for event_ix, sentence_ix in enumerate(sentences_ixs):
    print('Event:')
    print(events[key_events.events_mapping_list[event_ix]])
    print('Nearest sentence in article:')
    print(doc_sents_list[sentence_ix])

Event:
Attempt saved. Andreas Pereira (Manchester United) left footed shot from the centre of the box is saved in the bottom right corner.
Nearest sentence in article:
Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.

Event:
Attempt missed. Nicolas Pépé (Arsenal) left footed shot from outside the box is high and wide to the left. Assisted by Bukayo Saka.
Nearest sentence in article:
Bukayo
Event:
Attempt saved. Nicolas Pépé (Arsenal) left footed shot from the centre of the box is saved in the bottom right corner. Assisted by Pierre-Emerick Aubameyang.
Nearest sentence in article:
Arsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.
Event:
Attempt blocked. Paul Pogba (Manchester United) right footed shot from outside the box is blocked.
Nearest sentence in article:
Manchester United and Arsenal played out a grim

In [59]:
selected_article_sentences = sorted(list(set(sentences_ixs)))

In [60]:
selected_article_sentences

[0, 1, 2, 3, 4, 5, 13, 21, 34, 36]

In [163]:
# Solo con goles
key_events.match_summary(prueba, print_relations=True, ngram_range=(1,2), strip_accents='unicode')

Number of original events: 125
Number of processed events: 3
Number of sentences in original article: 39
Number of sentences in summary: 2
Event:
Goal!  Manchester United 1, Arsenal 0. Scott McTominay (Manchester United) right footed shot from outside the box to the top left corner. Assisted by Marcus Rashford.
Nearest sentence in article:
In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.

Event:
Goal!  Manchester United 1, Arsenal 1. Pierre-Emerick Aubameyang (Arsenal) left footed shot from the centre of the box to the centre of the goal. Assisted by Bukayo Saka with a through ball.Goal awarded following VAR Review.
Nearest sentence in article:
Arsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.
Event:
VAR Decision: Goal  Manchester United 1-1 Arsenal (Pierre-Emerick Au

"In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.\nArsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish."

In [166]:
key_events.match_summary(prueba, print_relations=True, ngram_range=(1,2), strip_accents='unicode')

Number of original events: 125
Number of processed events: 27
Number of sentences in original article: 39
Number of sentences in summary: 10
Event:
Attempt saved. Andreas Pereira (Manchester United) left footed shot from the centre of the box is saved in the bottom right corner.
Nearest sentence in article:
Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.

Event:
Attempt missed. Nicolas Pépé (Arsenal) left footed shot from outside the box is high and wide to the left. Assisted by Bukayo Saka.
Nearest sentence in article:
Bukayo
Event:
Attempt saved. Nicolas Pépé (Arsenal) left footed shot from the centre of the box is saved in the bottom right corner. Assisted by Pierre-Emerick Aubameyang.
Nearest sentence in article:
Arsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.
Event:
Attempt blocked. Paul Pogba (Manch

"Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.\nIn a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.\nArsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.It was originally ruled out for offside but the video assistant referee confirmed the striker had been played onside by Harry Maguire.\nGoalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford's late free-kick, while Bukayo Saka's goalbound shot crucially struck Victor Lindelof and flew over the top.\nMcTominay also headed a great chance well over from Ashley Young's corner but neither side could force a winner.\nUnited had the edge on chances, altho

## Para todos los partidos

In [212]:
key_events = KeyEventsSummary()

In [213]:
%%time
pd_df = key_events.run()

Number of original events: 108
Number of processed events: 40
Number of sentences in original article: 12
Number of sentences in summary: 6
Number of original events: 106
Number of processed events: 26
Number of sentences in original article: 6
Number of sentences in summary: 4
Number of original events: 105
Number of processed events: 26
Number of sentences in original article: 6
Number of sentences in summary: 4
Number of original events: 121
Number of processed events: 35
Number of sentences in original article: 5
Number of sentences in summary: 3
Number of original events: 116
Number of processed events: 30
Number of sentences in original article: 5
Number of sentences in summary: 5
Number of original events: 118
Number of processed events: 37
Number of sentences in original article: 5
Number of sentences in summary: 4
Number of original events: 117
Number of processed events: 28
Number of sentences in original article: 11
Number of sentences in summary: 8
Number of original events

/home/carlos/PycharmProjects/tfm/scripts/text/key_events_summary.py:127: UserWarning: Could not perform tfidf
  warnings.warn('Could not perform tfidf')


Number of original events: 103
Number of processed events: 25
Number of sentences in original article: 12
Number of sentences in summary: 4
Number of original events: 100
Number of processed events: 33
Number of sentences in original article: 8
Number of sentences in summary: 3
Number of original events: 114
Number of processed events: 21
Number of sentences in original article: 6
Number of sentences in summary: 3
Number of original events: 112
Number of processed events: 40
Number of sentences in original article: 7
Number of sentences in summary: 3
Number of original events: 125
Number of processed events: 28
Could not perform summary for https://www.bbc.com/sport/football/50693021


/home/carlos/PycharmProjects/tfm/scripts/text/key_events_summary.py:127: UserWarning: Could not perform tfidf
  warnings.warn('Could not perform tfidf')


Number of original events: 122
Number of processed events: 23
Could not perform summary for https://www.bbc.com/sport/football/50693028
Number of original events: 111
Number of processed events: 26
Number of sentences in original article: 6
Number of sentences in summary: 5
Number of original events: 91
Number of processed events: 15
Number of sentences in original article: 8
Number of sentences in summary: 3
Number of original events: 119
Number of processed events: 38
Number of sentences in original article: 12
Number of sentences in summary: 7
Number of original events: 118
Number of processed events: 31
Number of sentences in original article: 7
Number of sentences in summary: 4
Number of original events: 130
Number of processed events: 22
Number of sentences in original article: 16
Number of sentences in summary: 1
Number of original events: 108
Number of processed events: 30
Number of sentences in original article: 17
Number of sentences in summary: 4
Number of original events: 1

...

Number of original events: 116
Number of processed events: 17
Could not perform summary for https://www.bbc.com/sport/football/50537859
Number of original events: 107
Number of processed events: 24
Could not perform summary for https://www.bbc.com/sport/football/50537866
Number of original events: 106
Number of processed events: 26
Could not perform summary for https://www.bbc.com/sport/football/50530546
Number of original events: 80
Number of processed events: 21
Could not perform summary for https://www.bbc.com/sport/football/50530553
Number of original events: 92
Number of processed events: 30
Could not perform summary for https://www.bbc.com/sport/football/50530560
Number of original events: 87
Number of processed events: 22
Could not perform summary for https://www.bbc.com/sport/football/50530567
Number of original events: 117
Number of processed events: 26
Could not perform summary for https://www.bbc.com/sport/football/50530574
Number of original events: 109
Number of processed 

/home/carlos/PycharmProjects/tfm/scripts/text/key_events_summary.py:127: UserWarning: Could not perform tfidf
  warnings.warn('Could not perform tfidf')


Number of original events: 100
Number of processed events: 24
Could not perform summary for https://www.bbc.com/sport/football/50281875
Number of original events: 96
Number of processed events: 23
Could not perform summary for https://www.bbc.com/sport/football/50281882
Number of original events: 103
Number of processed events: 18
Could not perform summary for https://www.bbc.com/sport/football/50274991
Number of original events: 95
Number of processed events: 20
Could not perform summary for https://www.bbc.com/sport/football/50274998
Number of original events: 108
Number of processed events: 17
Could not perform summary for https://www.bbc.com/sport/football/50275005
Number of original events: 106
Number of processed events: 23
Could not perform summary for https://www.bbc.com/sport/football/50275012
Number of original events: 91
Number of processed events: 32
Could not perform summary for https://www.bbc.com/sport/football/50275026
Number of original events: 99
Number of processed e

/home/carlos/PycharmProjects/tfm/scripts/text/key_events_summary.py:127: UserWarning: Could not perform tfidf
  warnings.warn('Could not perform tfidf')


Number of original events: 101
Number of processed events: 15
Number of sentences in original article: 6
Number of sentences in summary: 4
Number of original events: 94
Number of processed events: 15
Number of sentences in original article: 9
Number of sentences in summary: 4
Number of original events: 87
Number of processed events: 24
Number of sentences in original article: 9
Number of sentences in summary: 5
Number of original events: 74
Number of processed events: 11
Number of sentences in original article: 11
Number of sentences in summary: 5
Number of original events: 90
Number of processed events: 23
Number of sentences in original article: 5
Number of sentences in summary: 4
Number of original events: 99
Number of processed events: 22
Number of sentences in original article: 9
Number of sentences in summary: 3
Number of original events: 78
Number of processed events: 18
Number of sentences in original article: 9
Number of sentences in summary: 5
Number of original events: 137
N

In [211]:
pd_df

,season_file,match_url,summary,article_sentences_ix,events_mapping
0,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,"[0, 0, 0, 1, 6, 6, 2, 2, 6, 0, 3, 2, 6, 6, 1, ...","[2, 4, 5, 6, 8, 11, 13, 14, 16, 17, 21, 28, 29..."


In [197]:
pd_df[pd_df.match_url=='https://www.bbc.com/sport/football/49871134']

,season_file,match_url,summary,article_sentences_ix,events_mapping
0,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,0,2
1,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,0,4
2,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,0,5
3,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,1,6
4,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,6,8
5,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,6,11
6,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,2,13
7,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,2,14
8,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,6,16
9,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,0,17


In [196]:
pd_df.dtypes

season_file             object
match_url               object
summary                 object
article_sentences_ix     int64
events_mapping           int64
dtype: object